In [1]:
import subprocess
import time
import datetime
from twilio.rest import Client
from twilio_info import *
import logging 
import google.cloud.logging
import re


# Setup Logging
file_name = ("Ping Test")
logger = logging.getLogger(__name__)
logging.basicConfig(format="%(filename)s:%(lineno)s - %(funcName)s() %(message)s")
logger.setLevel(logging.INFO)

auth_file = "VirtualToolbox-5f6ecffdcf58.json"

client = google.cloud.logging.Client.from_service_account_json(auth_file)
client.setup_logging()


# Setup Twilio

client = Client(twilio_sid, twilio_auth_token)

def send_alert(msg):
    dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
    msg_body = "{}:  {}".format(dtnow, msg)
    try:
        logger.error(msg_body)
        message = client.messages.create(body=msg_body,from_=phone_num_from,to=phone_num_to)
        time.sleep(30)
        logger.info("Attemped to send alert text.  Status:  {}".format(message.status))
    except:
        logger.critical("Attemped to send alert text.  Status:  Failed")
 

# Setup Ping script

def ping_addr(ip_address):
    try:
        res = subprocess.call(['ping', '-c', '3', ip_address])
    except:
        res = 99
    dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
    if res == 0:
        print ("{} - ping to {} OK".format(dtnow, ip_address))
    elif res == 2:
        print ("{} - no response from {}".format(dtnow, ip_address))
    else:
        print ("{} - ping to {} failed!".format(dtnow, ip_address))
    return(res)
    

class address:
    def __init__(self, ip_address, mac_address):
        self.call_count = 0
        self.ip_address = ip_address
        self.mac_address = mac_address
        if not self.verify_address(ip_address, mac_address):
            self.ip_address = self.find_ip_address(mac_address)
    
    def find_mac_address(self, ip_address):
        ping_addr(ip_address)
        pid = subprocess.Popen(["arp", "-n", ip_address], stdout=subprocess.PIPE)
        s = pid.communicate()[0]
        mac = re.search(r"(([a-f\d]{1,2}\:){5}[a-f\d]{1,2})", str(s)).groups()[0]
        return(mac)

    def find_ip_address(self, mac_address):
        ping_addr('255.255.255.255')
        pid = subprocess.Popen(["arp", "-an"], stdout=subprocess.PIPE)
        s = pid.communicate()[0]
        ip_address = re.search(f"\(([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)\) at {mac_address.lower()}", str(s).lower())[1]
        return(ip_address)

    def verify_address(self, ip_address, mac_address):
        mac_address2 = self.find_mac_address(ip_address)
        return(mac_address.lower() == mac_address2.lower())
    
    def get_ip_address(self):
        self.call_count += 1
        if self.call_count > 10:
            self.__init__(self.ip_address, self.mac_address)
        return(self.ip_address)
    
def main(ip_address, mac_address):    
    addr = address(ip_address, mac_address)
    while True:
        alert_count = 0

        while alert_count < 4:
            status = ping_addr(addr.get_ip_address())
            count = 0

            while (status != 0)&(count<5):
                dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
                print("{} - Ping failed, backoff and retry".format(dtnow))
                time.sleep(3)
                status = ping_addr(ip_address)
                count += 1

            if status != 0:
                send_alert("Host {} is down!".format(ip_address))
                alert_count += 1

            time.sleep(30)

        if alert_count >= 4:
            dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
            print("{} - Exceeded alert count, backing off for 4 hours".format(dtnow))
            time.sleep(4*60*60)
            count = 0
            alert_count = 0

logger.info("Starting Ping Test")



ip_address = "10.0.2.217"
mac_address = '58:EF:68:EA:54:2D'

main(ip_address, mac_address)


<ipython-input-1-e20adef88605>:117 - <module>() Starting Ping Test
Starting Ping Test


2022-08-15 07:43 - ping to 10.0.2.217 OK
2022-08-15 07:43 - ping to 10.0.2.217 OK


KeyboardInterrupt: 

In [2]:
main(ip_address, mac_address)

AttributeError: 'address' object has no attribute 'ping_addr'

In [9]:
addr = address(ip_address, mac_address)

2022-08-15 07:28 - ping to 10.0.2.217 OK


In [24]:
print(addr.call_count, ":  ", addr.get_ip_address())

2 :   10.0.2.217


In [7]:
verify_address(ip_address, mac_address)

get_ip_address(mac_address)

2022-08-15 07:17 - ping to 10.0.2.217 OK


True

In [3]:
arp_res = subprocess.call(['arp', mac_address])

In [4]:


get_mac_address(address)

'58:ef:68:ea:54:2d'

In [24]:
ping_addr('255.255.255.255')

2022-08-15 07:05 - ping to 255.255.255.255 OK


0

In [26]:


get_ip_address(mac_address)

2022-08-15 07:06 - ping to 255.255.255.255 OK


'10.0.2.217'

In [6]:
s

b'? (10.0.2.1) at d4:8c:b5:ac:98:b6 on en0 ifscope [ethernet]\n? (10.0.2.11) at 0:0:5e:0:1:1 on en0 ifscope [ethernet]\n? (10.0.2.154) at 3c:7:54:45:bb:c2 on en0 ifscope permanent [ethernet]\n? (10.0.2.157) at ac:ae:19:a4:55:fd on en0 ifscope [ethernet]\n? (10.0.2.158) at b8:27:eb:95:68:c3 on en0 ifscope [ethernet]\n? (10.0.2.159) at 8:5:81:2a:e6:12 on en0 ifscope [ethernet]\n? (10.0.2.166) at 2c:b4:3a:1e:1f:86 on en0 ifscope [ethernet]\n? (10.0.2.168) at 0:17:88:68:46:a4 on en0 ifscope [ethernet]\n? (10.0.2.182) at d0:4d:2c:3a:8a:5d on en0 ifscope [ethernet]\n? (10.0.2.186) at fe:ba:99:ef:40:93 on en0 ifscope [ethernet]\n? (10.0.2.208) at 6e:c:df:54:8:9d on en0 ifscope [ethernet]\n? (10.0.2.213) at 5e:63:ab:97:9:c4 on en0 ifscope [ethernet]\n? (10.0.2.217) at 58:ef:68:ea:54:2d on en0 ifscope [ethernet]\n? (10.0.2.255) at ff:ff:ff:ff:ff:ff on en0 ifscope [ethernet]\n? (169.254.255.255) at d4:8c:b5:ac:98:b6 on en1 [ethernet]\n? (224.0.0.251) at 1:0:5e:0:0:fb on en0 ifscope permanent [et

'10.0.2.217'

In [9]:

logger.critical("Attemped to send alert text.  Status:  Failed")

<ipython-input-9-9e23b7140649>:1 - <module>() Attemped to send alert text.  Status:  Failed
Attemped to send alert text.  Status:  Failed


In [2]:
   
send_alert("Host {} is down!".format(address))

<ipython-input-1-4bcc35ba3608>:29 - send_alert() 2020-07-08 07:59:  Host 10.0.2.183 is down!
2020-07-08 07:59:  Host 10.0.2.183 is down!
http_client.py:73 - request() POST Request: https://api.twilio.com/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages.json
POST Request: https://api.twilio.com/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages.json
http_client.py:75 - request() PAYLOAD: {'To': '13038182403', 'From': '12052933447', 'Body': '2020-07-08 07:59:  Host 10.0.2.183 is down!'}
PAYLOAD: {'To': '13038182403', 'From': '12052933447', 'Body': '2020-07-08 07:59:  Host 10.0.2.183 is down!'}
http_client.py:91 - request() POST Response: 201 {"sid": "SM25aad3d204d241109907399723350c9f", "date_created": "Wed, 08 Jul 2020 13:59:47 +0000", "date_updated": "Wed, 08 Jul 2020 13:59:47 +0000", "date_sent": null, "account_sid": "AC4ec8a19b2be6797eb5825903ff609d39", "to": "+13038182403", "from": "+12052933447", "messaging_service_sid": null, "body": "2020-07-08 07:59: 

In [5]:
message.status

NameError: name 'message' is not defined